# Yahtzee

In [1]:
import os
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import tensorflow as tf

from tensorflow import keras as tfk
from tensorflow.keras.callbacks import TensorBoard

%matplotlib inline

__MODEL_PATH = 'models'
__TENSOR_LOG_DIR = 'logs'

## Dataset

Let's start with looking at the provided dataset:

In [2]:
df = pd.read_csv('yahtzee-dataset.csv')
print('Labels:', df.label.unique())

df.head(10)

Labels: ['nothing' 'small-straight' 'three-of-a-kind' 'large-straight'
 'full-house' 'four-of-a-kind' 'yathzee']


,dice1,dice2,dice3,dice4,dice5,label
0,3,6,6,2,5,nothing
1,3,6,1,3,4,nothing
2,2,2,5,5,3,nothing
3,1,3,6,6,1,nothing
4,1,4,6,3,5,small-straight
5,4,1,4,3,1,nothing
6,4,4,4,6,2,three-of-a-kind
7,3,2,5,6,3,nothing
8,3,4,3,6,2,nothing
9,3,3,1,5,4,nothing


In order to classify these categorical labels, we have to 'one-hot encode' them:

In [3]:
one_hot_df = pd.get_dummies(df, prefix=['label'])
one_hot_df.head(10)

,dice1,dice2,dice3,dice4,dice5,label_four-of-a-kind,label_full-house,label_large-straight,label_nothing,label_small-straight,label_three-of-a-kind,label_yathzee
0,3,6,6,2,5,0,0,0,1,0,0,0
1,3,6,1,3,4,0,0,0,1,0,0,0
2,2,2,5,5,3,0,0,0,1,0,0,0
3,1,3,6,6,1,0,0,0,1,0,0,0
4,1,4,6,3,5,0,0,0,0,1,0,0
5,4,1,4,3,1,0,0,0,1,0,0,0
6,4,4,4,6,2,0,0,0,0,0,1,0
7,3,2,5,6,3,0,0,0,1,0,0,0
8,3,4,3,6,2,0,0,0,1,0,0,0
9,3,3,1,5,4,0,0,0,1,0,0,0


Before we can train any model, we have to split the data and the labels into X and Y:

In [4]:
shuffled = one_hot_df.sample(frac=1.)
X = shuffled.iloc[:,:5].copy()
Y = shuffled.iloc[:,5:].copy()

X.head(5)

,dice1,dice2,dice3,dice4,dice5
5754,2,6,1,4,2
5200,2,1,3,6,4
4108,5,5,1,2,6
459,3,6,1,1,3
4463,2,4,2,6,3


We also split the dataset into a 9:1 split for training and validating the model:

In [5]:
split = int(len(X.index) * .9)
X_train = X.iloc[:split]
X_valid = X.iloc[split:]
Y_train = Y.iloc[:split]
Y_valid = Y.iloc[split:]

print('Split X:', X_train.shape, X_valid.shape)
print('Split Y:', Y_train.shape, Y_valid.shape)

Y_train.head(5)

Split X: (5248, 5) (584, 5)
Split Y: (5248, 7) (584, 7)


,label_four-of-a-kind,label_full-house,label_large-straight,label_nothing,label_small-straight,label_three-of-a-kind,label_yathzee
5754,0,0,0,1,0,0,0
5200,0,0,0,0,1,0,0
4108,0,0,0,1,0,0,0
459,0,0,0,1,0,0,0
4463,0,0,0,1,0,0,0


In [6]:
def get_batch(data, labels, batch_size):
    x_batch = data.sample(frac=batch_size / len(data.index))
    return x_batch, labels.loc[x_batch.index]

## Models
We designed several models:

rank | name | layers | score
--- | --- | --- | ---
1 | model_6 | Dense(32, tanh), Dense(256, tanh), Dropout(0.1), Dense(256, tanh), Dropout(0.1), Dense(128, tanh), Dense(512, tanh), Dropout(0.1), Dense(512, tanh), Dropout(0.5), Dense(128, tanh)| 0.691780821918
2 | model_3 | Dense(32, ReLu), Dense(64, ReLu), Dropout(0.1), Dense(128, ReLu), Dropout(0.1), Dense(512, ReLu)| 0.683219178082
3 | model_4 | Dense(1024, tanh), Dense(512, tanh), Dense(128, tanh) | 0.679794520548
4 | model_2 | Dense(128, ReLu), Dense(256, ReLu), Dense(32, ReLu) | 0.676369863014
5 | model_1 | Dense(128, Sigmoid) | 0.674657534247
6 | model_5 | Dense(16, Sigmoid), Dense(32, Sigmoid | 0.674657534247

Dropout has a positive effect on the score as can be seen in the table. We also found that the tanh activation function performed well. 

In [7]:
def model_1(x, output_shape):
    """
    Single hidden layer with 128 neurons and Sigmoid activation function.
    """
    l_1 = tf.layers.dense(x, units=128, activation=tf.nn.sigmoid)
    return tf.layers.dense(l_1, units=output_shape, activation=None)

In [8]:
def model_2(x, output_shape):
    """
    Three hidden layers with different amounts of neurons and relu activation functions.
    """
    l_1 = tf.layers.dense(x, units=128, activation=tf.nn.relu)
    l_2 = tf.layers.dense(l_1, units=64, activation=tf.nn.relu)
    l_3 = tf.layers.dense(l_2, units=32, activation=tf.nn.relu)
    return tf.layers.dense(l_3, units=output_shape, activation=None)

In [9]:
def model_3(x, output_shape):
    """
    Six hidden layers with different amounts of neurons and 
    relu activation functions and 2 dropout layers.
    """
    l_1 = tf.layers.dense(x, units=12, activation=tf.nn.relu)
    l_2 = tf.layers.dense(l_1, units=24, activation=tf.nn.relu)
    l_3 = tf.layers.dense(l_2, units=48, activation=tf.nn.relu)
    d_3 = tf.layers.dropout(l_3, rate=.1)
    l_4 = tf.layers.dense(d_3, units=96, activation=tf.nn.relu)
    return tf.layers.dense(l_4, units=output_shape, activation=None)

In [10]:
def model_4(x, output_shape):
    """
    Three hidden layers with different amounts of neurons and relu activation functions.
    """
    l_1 = tf.layers.dense(x, units=128, activation=tf.nn.tanh)
    l_2 = tf.layers.dense(l_1, units=64, activation=tf.nn.tanh)
    l_3 = tf.layers.dense(l_2, units=32, activation=tf.nn.tanh)
    return tf.layers.dense(l_3, units=output_shape, activation=None)

In [11]:
def model_5(x, output_shape):
    """
    High number of neurons in layers, decreasing per layer
    """
    l_1 = tf.layers.dense(x, units=1000, activation=tf.nn.tanh)
    l_2 = tf.layers.dense(l_1, units=500, activation=tf.nn.tanh)
    l_3 = tf.layers.dense(l_2, units=250, activation=tf.nn.tanh)
    return tf.layers.dense(l_3, units=output_shape, activation=None)

In [12]:
def model_6(x, output_shape):
    """
    High number of neurons in layers, increasing per layer
    """
    l_1 = tf.layers.dense(x, units=250, activation=tf.nn.tanh)
    l_2 = tf.layers.dense(l_1, units=500, activation=tf.nn.tanh)
    l_3 = tf.layers.dense(l_2, units=1000, activation=tf.nn.tanh)
    return tf.layers.dense(l_3, units=output_shape, activation=None)

In [13]:
def model_7(x, output_shape):
    """
    """
    l_1 = tf.layers.dense(x, units=64, activation=tf.nn.relu)
    l_2 = tf.layers.dense(l_1, units=128, activation=tf.nn.relu)
    l_3 = tf.layers.dense(l_2, units=256, activation=tf.nn.relu)
    l_4 = tf.layers.dense(l_3, units=512, activation=tf.nn.relu)
    d_4 = tf.layers.dropout(l_4, rate=.3)
    l_5 = tf.layers.dense(d_4, units=64, activation=tf.nn.relu)
    return tf.layers.dense(l_5, units=output_shape, activation=None)

In [14]:
def model_8(x, output_shape):
    """
    """
    l_1 = tf.layers.dense(x, units=64, activation=tf.nn.relu)
    d_1 = tf.layers.dropout(l_1, rate=.0)
    l_2 = tf.layers.dense(d_1, units=128, activation=tf.nn.relu)
    d_2 = tf.layers.dropout(l_2, rate=.1)
    l_3 = tf.layers.dense(d_2, units=256, activation=tf.nn.relu)
    d_3 = tf.layers.dropout(l_3, rate=.2)
    l_4 = tf.layers.dense(d_3, units=512, activation=tf.nn.relu)
    d_4 = tf.layers.dropout(l_4, rate=.3)
    l_5 = tf.layers.dense(d_4, units=64, activation=tf.nn.relu)
    return tf.layers.dense(l_5, units=output_shape, activation=None)

We start with the placeholder for our 5-dice input and 7-class output and choose a model:

In [15]:
x = tf.placeholder(tf.float32, shape=[None, X.shape[1]], name='x')
y = tf.placeholder(tf.float32, shape=[None, Y.shape[1]], name='y')

model_fn = model_8
y_pred = model_fn(x, Y.shape[1])

## Training

We choose an optimizer, a loss functon and metrics:

In [16]:
# Loss function
cross_entropy = tf.nn.softmax_cross_entropy_with_logits_v2(labels=y, logits=y_pred)
loss_fn = tf.reduce_mean(cross_entropy)

# Optimizer minimizes the loss
optimizer = tf.train.AdamOptimizer(learning_rate=.001).minimize(loss_fn)

# Accuracy metric
#   checks if the indices of the highest values in the real 
#   and predicted arrays are equal
prediction = tf.equal(tf.argmax(y, axis=1), tf.argmax(y_pred, axis=1))
accuracy = tf.reduce_mean(tf.cast(prediction, tf.float32))

We train the model using a certain batch size and for a number of iterations while posting scalars to TensorBoard:

In [17]:
iters = 3000
batch_size = 200

session = tf.Session()
session.run(tf.global_variables_initializer())

tf.summary.scalar('loss', loss_fn)
tf.summary.scalar('acc', accuracy)
summary = tf.summary.merge_all()
writer = tf.summary.FileWriter(os.path.join(__TENSOR_LOG_DIR, model_fn.__name__), session.graph)

for i in range(iters):
    x_batch, y_batch = get_batch(X_train, Y_train, batch_size)
    loss_val, _, acc_val, summary_res = session.run([loss_fn, optimizer, accuracy, summary], 
                                          feed_dict={x: x_batch, y: y_batch})

    writer.add_summary(summary_res, global_step=i)    
    print('Training - i:', i+1, 'Loss:', loss_val, 'Accuracy:', acc_val)

    # Validate every 50 iterations
    if i % 50 == 0:
        x_batch, y_batch = get_batch(X_valid, Y_valid, batch_size)
        acc_val = session.run([accuracy], feed_dict={x: x_batch, y: y_batch})
        print('Validation - i:', i+1, ' Accuracy:', acc_val)

Training - i: 1 Loss: 2.192661 Accuracy: 0.02
Validation - i: 1  Accuracy: [0.69]
Training - i: 2 Loss: 1.7579706 Accuracy: 0.63
Training - i: 3 Loss: 1.4803977 Accuracy: 0.69
Training - i: 4 Loss: 1.3888452 Accuracy: 0.61
Training - i: 5 Loss: 1.2159992 Accuracy: 0.675
Training - i: 6 Loss: 1.1466622 Accuracy: 0.69
Training - i: 7 Loss: 1.41533 Accuracy: 0.63
Training - i: 8 Loss: 1.1347055 Accuracy: 0.69
Training - i: 9 Loss: 1.2810593 Accuracy: 0.615
Training - i: 10 Loss: 1.135078 Accuracy: 0.64
Training - i: 11 Loss: 1.1256759 Accuracy: 0.695
Training - i: 12 Loss: 1.0870862 Accuracy: 0.645
Training - i: 13 Loss: 1.1492213 Accuracy: 0.655
Training - i: 14 Loss: 1.0986248 Accuracy: 0.665
Training - i: 15 Loss: 1.009088 Accuracy: 0.705
Training - i: 16 Loss: 0.9887791 Accuracy: 0.73
Training - i: 17 Loss: 1.2981027 Accuracy: 0.575
Training - i: 18 Loss: 1.0644974 Accuracy: 0.7
Training - i: 19 Loss: 1.0845075 Accuracy: 0.68
Training - i: 20 Loss: 1.0568337 Accuracy: 0.685
Training -

Training - i: 167 Loss: 1.0733719 Accuracy: 0.64
Training - i: 168 Loss: 0.95824945 Accuracy: 0.67
Training - i: 169 Loss: 1.0354418 Accuracy: 0.645
Training - i: 170 Loss: 0.9547444 Accuracy: 0.665
Training - i: 171 Loss: 1.0507964 Accuracy: 0.66
Training - i: 172 Loss: 0.9085906 Accuracy: 0.685
Training - i: 173 Loss: 1.1529697 Accuracy: 0.625
Training - i: 174 Loss: 0.9295259 Accuracy: 0.68
Training - i: 175 Loss: 1.0066146 Accuracy: 0.66
Training - i: 176 Loss: 0.9916388 Accuracy: 0.665
Training - i: 177 Loss: 0.94854224 Accuracy: 0.66
Training - i: 178 Loss: 1.0641519 Accuracy: 0.665
Training - i: 179 Loss: 0.91304564 Accuracy: 0.695
Training - i: 180 Loss: 1.0520914 Accuracy: 0.665
Training - i: 181 Loss: 0.86249316 Accuracy: 0.7
Training - i: 182 Loss: 1.0112658 Accuracy: 0.685
Training - i: 183 Loss: 0.9611797 Accuracy: 0.685
Training - i: 184 Loss: 1.0820553 Accuracy: 0.605
Training - i: 185 Loss: 0.90476346 Accuracy: 0.7
Training - i: 186 Loss: 0.9225441 Accuracy: 0.69
Traini

Training - i: 334 Loss: 0.81539255 Accuracy: 0.72
Training - i: 335 Loss: 0.82162726 Accuracy: 0.77
Training - i: 336 Loss: 0.9016156 Accuracy: 0.7
Training - i: 337 Loss: 0.82832694 Accuracy: 0.69
Training - i: 338 Loss: 0.8240779 Accuracy: 0.745
Training - i: 339 Loss: 0.89304304 Accuracy: 0.69
Training - i: 340 Loss: 0.85470843 Accuracy: 0.68
Training - i: 341 Loss: 0.8787743 Accuracy: 0.685
Training - i: 342 Loss: 0.8028795 Accuracy: 0.725
Training - i: 343 Loss: 0.83281344 Accuracy: 0.73
Training - i: 344 Loss: 0.82390755 Accuracy: 0.73
Training - i: 345 Loss: 0.86113197 Accuracy: 0.7
Training - i: 346 Loss: 0.81306845 Accuracy: 0.715
Training - i: 347 Loss: 0.7019093 Accuracy: 0.795
Training - i: 348 Loss: 0.9795627 Accuracy: 0.645
Training - i: 349 Loss: 0.86348146 Accuracy: 0.705
Training - i: 350 Loss: 0.866639 Accuracy: 0.705
Training - i: 351 Loss: 0.8624703 Accuracy: 0.68
Validation - i: 351  Accuracy: [0.725]
Training - i: 352 Loss: 0.69788843 Accuracy: 0.755
Training - i:

Training - i: 499 Loss: 0.7038064 Accuracy: 0.765
Training - i: 500 Loss: 0.80887926 Accuracy: 0.7
Training - i: 501 Loss: 0.81070787 Accuracy: 0.69
Validation - i: 501  Accuracy: [0.705]
Training - i: 502 Loss: 0.6114567 Accuracy: 0.8
Training - i: 503 Loss: 0.6338581 Accuracy: 0.765
Training - i: 504 Loss: 0.70519686 Accuracy: 0.765
Training - i: 505 Loss: 0.6685213 Accuracy: 0.78
Training - i: 506 Loss: 0.6805468 Accuracy: 0.755
Training - i: 507 Loss: 0.8068964 Accuracy: 0.69
Training - i: 508 Loss: 0.6132643 Accuracy: 0.785
Training - i: 509 Loss: 0.6398418 Accuracy: 0.765
Training - i: 510 Loss: 0.7825974 Accuracy: 0.705
Training - i: 511 Loss: 0.63337237 Accuracy: 0.755
Training - i: 512 Loss: 0.6403935 Accuracy: 0.785
Training - i: 513 Loss: 0.6187208 Accuracy: 0.795
Training - i: 514 Loss: 0.6879503 Accuracy: 0.78
Training - i: 515 Loss: 0.65581214 Accuracy: 0.765
Training - i: 516 Loss: 0.63339925 Accuracy: 0.76
Training - i: 517 Loss: 0.5630359 Accuracy: 0.81
Training - i: 5

Training - i: 666 Loss: 0.4864408 Accuracy: 0.82
Training - i: 667 Loss: 0.39032993 Accuracy: 0.84
Training - i: 668 Loss: 0.49640855 Accuracy: 0.81
Training - i: 669 Loss: 0.46478814 Accuracy: 0.795
Training - i: 670 Loss: 0.4752145 Accuracy: 0.805
Training - i: 671 Loss: 0.42010644 Accuracy: 0.85
Training - i: 672 Loss: 0.55807066 Accuracy: 0.785
Training - i: 673 Loss: 0.4559298 Accuracy: 0.825
Training - i: 674 Loss: 0.4202536 Accuracy: 0.825
Training - i: 675 Loss: 0.50721496 Accuracy: 0.78
Training - i: 676 Loss: 0.45355576 Accuracy: 0.83
Training - i: 677 Loss: 0.4396129 Accuracy: 0.82
Training - i: 678 Loss: 0.43884417 Accuracy: 0.825
Training - i: 679 Loss: 0.44921395 Accuracy: 0.805
Training - i: 680 Loss: 0.40139842 Accuracy: 0.845
Training - i: 681 Loss: 0.44949067 Accuracy: 0.825
Training - i: 682 Loss: 0.47627786 Accuracy: 0.825
Training - i: 683 Loss: 0.44344848 Accuracy: 0.83
Training - i: 684 Loss: 0.4092369 Accuracy: 0.85
Training - i: 685 Loss: 0.43245193 Accuracy: 0

...

Training - i: 2323 Loss: 0.057027988 Accuracy: 0.98
Training - i: 2324 Loss: 0.054284297 Accuracy: 0.98
Training - i: 2325 Loss: 0.0673471 Accuracy: 0.97
Training - i: 2326 Loss: 0.07521184 Accuracy: 0.97
Training - i: 2327 Loss: 0.04430933 Accuracy: 0.99
Training - i: 2328 Loss: 0.067303605 Accuracy: 0.975
Training - i: 2329 Loss: 0.0916249 Accuracy: 0.95
Training - i: 2330 Loss: 0.033273216 Accuracy: 0.995
Training - i: 2331 Loss: 0.04877286 Accuracy: 0.98
Training - i: 2332 Loss: 0.047022544 Accuracy: 0.98
Training - i: 2333 Loss: 0.061422262 Accuracy: 0.97
Training - i: 2334 Loss: 0.06067094 Accuracy: 0.97
Training - i: 2335 Loss: 0.12686974 Accuracy: 0.95
Training - i: 2336 Loss: 0.046262633 Accuracy: 0.99
Training - i: 2337 Loss: 0.06362248 Accuracy: 0.975
Training - i: 2338 Loss: 0.038744878 Accuracy: 0.99
Training - i: 2339 Loss: 0.07346422 Accuracy: 0.965
Training - i: 2340 Loss: 0.078418024 Accuracy: 0.95
Training - i: 2341 Loss: 0.05474741 Accuracy: 0.97
Training - i: 2342 L

Training - i: 2495 Loss: 0.07205653 Accuracy: 0.97
Training - i: 2496 Loss: 0.045909982 Accuracy: 0.985
Training - i: 2497 Loss: 0.08301763 Accuracy: 0.965
Training - i: 2498 Loss: 0.09424553 Accuracy: 0.95
Training - i: 2499 Loss: 0.058667604 Accuracy: 0.975
Training - i: 2500 Loss: 0.06771775 Accuracy: 0.96
Training - i: 2501 Loss: 0.047848433 Accuracy: 0.985
Validation - i: 2501  Accuracy: [0.905]
Training - i: 2502 Loss: 0.14573498 Accuracy: 0.93
Training - i: 2503 Loss: 0.050269447 Accuracy: 0.99
Training - i: 2504 Loss: 0.10581978 Accuracy: 0.94
Training - i: 2505 Loss: 0.022125497 Accuracy: 1.0
Training - i: 2506 Loss: 0.0920718 Accuracy: 0.95
Training - i: 2507 Loss: 0.09076008 Accuracy: 0.965
Training - i: 2508 Loss: 0.08000234 Accuracy: 0.98
Training - i: 2509 Loss: 0.09155284 Accuracy: 0.96
Training - i: 2510 Loss: 0.063246205 Accuracy: 0.965
Training - i: 2511 Loss: 0.07904118 Accuracy: 0.965
Training - i: 2512 Loss: 0.065322585 Accuracy: 0.98
Training - i: 2513 Loss: 0.087

Training - i: 2653 Loss: 0.044915903 Accuracy: 0.975
Training - i: 2654 Loss: 0.084543 Accuracy: 0.96
Training - i: 2655 Loss: 0.07274019 Accuracy: 0.96
Training - i: 2656 Loss: 0.04169903 Accuracy: 0.985
Training - i: 2657 Loss: 0.07592577 Accuracy: 0.965
Training - i: 2658 Loss: 0.0750506 Accuracy: 0.965
Training - i: 2659 Loss: 0.046818294 Accuracy: 0.995
Training - i: 2660 Loss: 0.08321939 Accuracy: 0.975
Training - i: 2661 Loss: 0.055153526 Accuracy: 0.985
Training - i: 2662 Loss: 0.03746498 Accuracy: 0.985
Training - i: 2663 Loss: 0.048169736 Accuracy: 0.98
Training - i: 2664 Loss: 0.07374696 Accuracy: 0.98
Training - i: 2665 Loss: 0.058192376 Accuracy: 0.98
Training - i: 2666 Loss: 0.057615146 Accuracy: 0.98
Training - i: 2667 Loss: 0.054760683 Accuracy: 0.975
Training - i: 2668 Loss: 0.065000355 Accuracy: 0.98
Training - i: 2669 Loss: 0.083533905 Accuracy: 0.975
Training - i: 2670 Loss: 0.06044894 Accuracy: 0.965
Training - i: 2671 Loss: 0.0680184 Accuracy: 0.97
Training - i: 2

Training - i: 2816 Loss: 0.06566493 Accuracy: 0.965
Training - i: 2817 Loss: 0.051261414 Accuracy: 0.98
Training - i: 2818 Loss: 0.060157374 Accuracy: 0.975
Training - i: 2819 Loss: 0.08445083 Accuracy: 0.97
Training - i: 2820 Loss: 0.07076442 Accuracy: 0.97
Training - i: 2821 Loss: 0.06714854 Accuracy: 0.97
Training - i: 2822 Loss: 0.06149304 Accuracy: 0.97
Training - i: 2823 Loss: 0.060419865 Accuracy: 0.96
Training - i: 2824 Loss: 0.07938059 Accuracy: 0.975
Training - i: 2825 Loss: 0.04935522 Accuracy: 0.985
Training - i: 2826 Loss: 0.07562218 Accuracy: 0.975
Training - i: 2827 Loss: 0.07353268 Accuracy: 0.97
Training - i: 2828 Loss: 0.05614607 Accuracy: 0.97
Training - i: 2829 Loss: 0.08401138 Accuracy: 0.96
Training - i: 2830 Loss: 0.061782107 Accuracy: 0.985
Training - i: 2831 Loss: 0.06323325 Accuracy: 0.965
Training - i: 2832 Loss: 0.08944269 Accuracy: 0.95
Training - i: 2833 Loss: 0.087014936 Accuracy: 0.955
Training - i: 2834 Loss: 0.06050848 Accuracy: 0.975
Training - i: 283

Training - i: 2975 Loss: 0.08380908 Accuracy: 0.95
Training - i: 2976 Loss: 0.047682285 Accuracy: 0.985
Training - i: 2977 Loss: 0.040169254 Accuracy: 0.995
Training - i: 2978 Loss: 0.047728315 Accuracy: 0.985
Training - i: 2979 Loss: 0.06360569 Accuracy: 0.99
Training - i: 2980 Loss: 0.04706991 Accuracy: 0.975
Training - i: 2981 Loss: 0.0693964 Accuracy: 0.965
Training - i: 2982 Loss: 0.038676575 Accuracy: 0.995
Training - i: 2983 Loss: 0.031966556 Accuracy: 0.985
Training - i: 2984 Loss: 0.040958427 Accuracy: 0.985
Training - i: 2985 Loss: 0.033823416 Accuracy: 0.99
Training - i: 2986 Loss: 0.055094272 Accuracy: 0.98
Training - i: 2987 Loss: 0.03183309 Accuracy: 0.99
Training - i: 2988 Loss: 0.045563813 Accuracy: 0.98
Training - i: 2989 Loss: 0.04978889 Accuracy: 0.985
Training - i: 2990 Loss: 0.035836913 Accuracy: 0.99
Training - i: 2991 Loss: 0.055332135 Accuracy: 0.98
Training - i: 2992 Loss: 0.031535674 Accuracy: 0.99
Training - i: 2993 Loss: 0.04679824 Accuracy: 0.975
Training -

We validate the model with the data it has not seen yet:

In [18]:
# Validate the model with unseen data
loss_val, _, acc_val = session.run([loss_fn, optimizer, accuracy], 
                                    feed_dict={x: X_valid, y: Y_valid})

# Print test metrics
print('Loss:', loss_val, 'Accuracy:', acc_val)

Loss: 0.26099378 Accuracy: 0.9058219


We save the model that worked best:

In [19]:
# model.save('{}.h5'.format(os.path.join(__MODEL_PATH, model_fn.__name__)))

We load the model that worked best:

In [20]:
# best_model_file_name = 'model_7.h5'
# best_model = tfk.models.load_model(os.path.join(__MODEL_PATH, best_model_file_name))

# print('Model {} retrieved!'.format(best_model_file_name))

# score = best_model.evaluate(X_valid, Y_valid)
# print('Loss:', score[0], 'Accuracy:', score[1])